## Mangrove detection

The application runs deep learning to detect mangrove trees

### <a name="service">Service Definition

In [25]:
service = dict([('title', 'ewf-ext-03-01-01 - Mangrove detection'),
                ('abstract', 'Image classification with deep learning for mangrove detection'),
                ('id', 'ewf-ext-03-01-01')])

### <a name="runtime">Runtime parameter definition

**Input identifier**

This is the Sentinel-2 product identifier

In [87]:
input_identifier = 'S2B_MSIL2A_20190609T112119_N0212_R037_T28PDT_20190609T144134'

**Input reference**

This is the Sentinel-2 catalogue reference

In [88]:
input_reference = 'https://catalog.terradue.com/sentinel2/search?uid=S2B_MSIL2A_20190609T112119_N0212_R037_T28PDT_20190609T144134'

**Data path**

This path defines where the data is staged-in. 

In [89]:
data_path = '/workspace/data'

In [90]:
my_data_path = data_path + "/" + input_identifier + "/" + input_identifier + ".SAFE"

### <a name="workflow">Workflow

#### Import the packages required for processing the Sentinel-1 backscatter

In [91]:
import os
import numpy as np
import gdal
import gc
from tensorflow.keras import models
import xml.etree.ElementTree as ET
import sys

os.environ['GDAL_DATA'] = '/opt/anaconda/share/gdal/'

# PIPELINE

#### Transform a 20m band into a 10m band

In [92]:
def transform_image(pixel_res, resampling_method, t4, t5):
    """
    Method to transform an image from one resolution to another

    Parameters
    ----------
    pixel_res : float
        resolution of the image to create
    resampling_method : string
        What method to use to transform the image
    t4 : string
        Location of the original file
    t5 : string
        Location of the final file

    Returns
    -------
    0/1.

    """

    #command = "/opt/anaconda/envs/mangrove_final/bin/gdalwarp -overwrite -tr {} {} -r {} {} {}".format(pixel_res,
     #                                                             pixel_res,
     #                                                             resampling_method,
     #                                                             t4,
     #                                                             t5)
    
    ds = gdal.Warp(srcDSOrSrcDSTab  = t4, destNameOrDestDS = t5, xRes=pixel_res, yRes=pixel_res, outputType=gdal.GDT_Int16)
    #return os.system(command)

#### Read the bands from the folder and transform them into a matrix

In [93]:
def read_bands_to_array(folder_location):
    """
    Read the necessary bands and returns an numpy array

    Parameters
    ----------
    folder_location : string
        Location of the folder with the sentinel 2 images

    Returns
    -------
    img_array : numpy array 3 dim [image_rows, image_cols, n_bands]
    """
    
    # save path of all images
    t_images = []
    
    # we will read the manifest to find the location of the bands that we want
    print(folder_location + "/manifest.safe")
    root = ET.parse(folder_location + "/manifest.safe").getroot()

    action = root.find("./dataObjectSection/dataObject/[@ID='IMG_DATA_Band_B03_10m_Tile1_Data']")
    for n in action.iter():
        if(n.tag == 'fileLocation'):
            t = n.attrib
            t1 = folder_location + t['href'][1:]
            
    action = root.find("./dataObjectSection/dataObject/[@ID='IMG_DATA_Band_B04_10m_Tile1_Data']")
    for n in action.iter():
        if(n.tag == 'fileLocation'):
            t = n.attrib
            t_images.append(folder_location + t['href'][1:])


    action = root.find("./dataObjectSection/dataObject/[@ID='IMG_DATA_Band_B08_10m_Tile1_Data']")
    for n in action.iter():
        if(n.tag == 'fileLocation'):
            t = n.attrib
            t_images.append(folder_location + t['href'][1:])
            
    action = root.find("./dataObjectSection/dataObject/[@ID='IMG_DATA_Band_B11_20m_Tile1_Data']")
    for n in action.iter():
        if(n.tag == 'fileLocation'):
            t = n.attrib
            t2 = folder_location + t['href'][1:]

    
    # add the path of the image that will be added
    t_images.append(folder_location + "/B11_10m.jp2")
      
     # open the first band to get the necessary proprieties to correctly convert the B11_20m into B11_10m  
    _temp_img = gdal.Open(t1)
    transf = _temp_img.GetGeoTransform()
    proj = _temp_img.GetProjection()
    
    print()
    print("GeoTransform: {}".format(transf))
    print("Projection: {}".format(proj))     
    print("Size is {} x {} x {}".format(_temp_img.RasterXSize,
                                    _temp_img.RasterYSize,
                                    _temp_img.RasterCount))
    
    # find first the original pixel size
    # http://www2.geog.ucl.ac.uk/~plewis/geogg122_local/geogg122-old/Chapter4_GDAL/GDAL_Python_bindings.html
    up_left_east_coord, ew_pixel_space, rot, up_left_north_coord, rot2, ns_pixel_spacing = _temp_img.GetGeoTransform()
    pixel_res = ew_pixel_space
    resampling_method = "near"

    transform_image(pixel_res, resampling_method, t2, t_images[2])



    # create the matrix to save the full image with the 4 bands, plus the 3 new indices that will be created
    # in computer vision the axis start on upper left conner 
    cols = _temp_img.RasterXSize
    rows = _temp_img.RasterYSize
    
    img_array = np.zeros((rows, cols, 4), dtype=np.float32)
    img_array[:,:, 0] = _temp_img.GetRasterBand(1).ReadAsArray()
    
    for img in range(0, len(t_images)):
        print("New band:", t_images[img])
        _temp_img = gdal.Open(t_images[img])
        img_array[:,:, img+1] = _temp_img.GetRasterBand(1).ReadAsArray()    


    
    # cleaning
    _temp_img = None    
    del _temp_img, t_images
    gc.collect() 
    
        


    return img_array, transf, proj

#### Create new indices based on the 4 bands

In [94]:
def create_indices(img_array):
    """
    From the bands create new indices

    Parameters
    ----------
    img_array : numpy array
    Returns
    -------
    final_img_array : numpy array
    """
    final_img_array = np.zeros((img_array.shape[0], img_array.shape[1], 3), dtype=np.float32)
    
    print("\nCreate indices")
    
    final_img_array[:, :, 0] = np.divide((img_array[:,:,1]-img_array[:,:,3]), (img_array[:,:,1]+img_array[:,:,3]))
    print("MNDWI created")

    final_img_array[:, :, 1] = np.divide((img_array[:,:,3]-img_array[:,:,2]),(img_array[:,:,3]+img_array[:,:,2]))
    print("NDVI created")
    
    # this matrix is no longer needed
    del img_array
    gc.collect()
    
    # https://stackoverflow.com/questions/26248654/how-to-return-0-with-divide-by-zero/40022737
    a = abs(final_img_array[:, :, 0])-abs(final_img_array[:, :, 1])
    b = abs(final_img_array[:, :, 0])+abs(final_img_array[:, :, 1])

    with np.errstate(divide='ignore', invalid='ignore'):
        c = np.divide(a, b)
        c[ ~ np.isfinite( c )] = 0  # -inf inf NaN

    final_img_array[:, :, 2] = c
    print("MMRI created")
    
    del a,b,c
    return final_img_array

## Normalize the data

In [95]:
def array_normalizer(data_array):
    # norm_mode 0 is for normalization between 0-1 and 1
    
    array_max = np.nanmax(data_array.astype(np.float32), axis=(0,1))
    array_min = np.nanmin(data_array.astype(np.float32), axis=(0,1)) # calculating min with no zeros
    
    # normalizing between 0 - 1
    data_array_norm = (data_array.astype(np.float32) - array_min.astype(np.float32))/(array_max.astype(np.float32) - array_min.astype(np.float32))

    return data_array_norm

#### Create the tiles

In [96]:
def array_cutter(data_arr, tile_size_x, tile_size_y):
    """
    This function cuts an array into different parts based on [tile_size_x, tile_size_y] and save them in nparray.

    Parameters
    ----------
    data_arr : numpy array in 3 dim
    tile_size_x : int
    tile_size_y : int

    Returns
    -------
    tiles_array : numpy array in 4 dim [n_tiles, 512, 512, n_bands]
    n_tiles_x : int
    n_tiles_y : int
    """

    # If the image has 2D array (e.g. 1980, 1980) converts it to 3D array (e.g. 1980, 1980, 1)
    if len(data_arr.shape) == 2:
          data_arr = data_arr[..., np.newaxis]

    # Replace Nane with 0.0000
    data_arr = np.nan_to_num(data_arr, nan = 0.0000)

    xsize, ysize, depth = data_arr.shape
    img_counter = 0
    
    # find the number of tiles and how many pixeis are left from the tiles dimensions provided
    n_tiles_x, pad_x = divmod(xsize, tile_size_x)
    n_tiles_y, pad_y = divmod(ysize, tile_size_y)
    
    # if there are pixeis left, we need to add extra tiles
    if(pad_x != 0): n_tiles_x += 1
    if(pad_y != 0): n_tiles_y += 1
    
    total_tiles = n_tiles_x*n_tiles_y
    
    # compute the necessary pads to complete the tiles with the left pixeis
    pad_x = tile_size_x - pad_x
    pad_y = tile_size_y - pad_y
    
    # 4D structure to save the tiles: n_tiles_total, xsize*ysize with depth bands plus one band for the prediction (to be added later)
    tiles_array = np.zeros((total_tiles, tile_size_x, tile_size_y, depth), dtype=np.float32)

    
    print("Horizontal tiles: ", n_tiles_x)
    print("Vertical tiles: ", n_tiles_y)
    print("Total tiles: ", n_tiles_x*n_tiles_y)
    print("pad x: ", pad_x)
    print("pad y: ", pad_y)
    print("Size of final structure: ", tiles_array.shape)

    for cntr_i, i in enumerate(range(0, xsize, tile_size_x)):
        for cntr_j, j in enumerate(range(0, ysize, tile_size_y)):

            gen_size = data_arr[ i:i+tile_size_x, j:j+tile_size_y,:].shape
            
            # test to see if the tile has the defined regular size
            if gen_size == (tile_size_x, tile_size_y, data_arr[ i:i+tile_size_x, j:j+tile_size_y,:].shape[2]):
                tiles_array[img_counter,:,:,:depth] = data_arr[i:i+tile_size_x, j:j+tile_size_y,:]
                img_counter += 1

            else:
                if(gen_size[0] != tile_size_x and gen_size[1] != tile_size_y):
                    tiles_array[img_counter,:,:,:depth] = np.pad(data_arr[i:i+tile_size_x, j:j+tile_size_y, :], pad_width = [(0, pad_x), (0, pad_y), (0, 0)], mode='reflect') # The mode can be ['constant', 'edge', 'wrap', 'reflect', 'symmetric']
                
                elif(gen_size[0] != tile_size_x):
                    tiles_array[img_counter,:,:,:depth] = np.pad(data_arr[i:i+tile_size_x, j:j+tile_size_y, :], pad_width = [(0, pad_x), (0, 0), (0, 0)], mode='reflect')
                    
                elif(gen_size[1] != tile_size_y):
                    tiles_array[img_counter,:,:,:depth] = np.pad(data_arr[i:i+tile_size_x, j:j+tile_size_y, :], pad_width = [(0, 0), (0, pad_y), (0, 0)], mode='reflect') # The mode can be ['constant', 'edge', 'wrap', 'reflect', 'symmetric']

                img_counter += 1
                    
    return tiles_array, n_tiles_x, n_tiles_y, pad_x, pad_y

#### Recreate the image and save it

In [97]:
def create_image(my_data_path, tiles_array, n_tiles_x, n_tiles_y, tile_size, dim_x, dim_y, n_bands, pad_x, pad_y, transf, proj):
    """
    From the numpy array wiht tiles, recreate the full image

    Parameters
    ----------
    tiles_array : numpy array
    n_tiles_x : int
    n_tiles_y : int
    tile_size : int
    dim_x : int
    dim_y : int
    n_bands : int
    transf : gdal object
        DESCRIPTION.
    proj : gdal object
    Returns
    -------
    None.

    """    
    
    array_to_write = np.zeros((dim_x+pad_x, dim_y+pad_y, n_bands), dtype=np.float32)
    p=0
    for x_pos in range(n_tiles_x):
        for y_pos in range(n_tiles_y):
            array_to_write[tile_size*x_pos:tile_size*x_pos+tile_size, tile_size*y_pos:tile_size*y_pos+tile_size, 0] = tiles_array[p,:,:]
            p+=1
    
    
    # array_to_write = np.reshape(tiles_array, (dim_y+pad_y, dim_x+pad_x, n_bands))
    print(array_to_write[:dim_x, :dim_y, :].shape)
    
    # replace the values with ints
    _temp = array_to_write.copy()
    _temp = np.where((_temp >= 0) & (_temp <= 0.1), 3, _temp)
    _temp = np.where((_temp > 0.1) & (_temp <= 0.2), 3, _temp)
    _temp = np.where((_temp > 0.2) & (_temp <= 0.3), 3, _temp)
    _temp = np.where((_temp > 0.3) & (_temp <= 0.4), 4, _temp)
    _temp = np.where((_temp > 0.4) & (_temp <= 0.5), 5, _temp)
    _temp = np.where((_temp > 0.5) & (_temp <= 0.6), 6, _temp)
    _temp = np.where((_temp > 0.6) & (_temp <= 0.7), 7, _temp)
    _temp = np.where((_temp > 0.7) & (_temp <= 0.8), 8, _temp)
    _temp = np.where((_temp > 0.8) & (_temp <= 0.9), 9, _temp)
    _temp = np.where((_temp > 0.9) & (_temp <= 1), 10, _temp)
    
    
    print("size: ", _temp.shape[0]*_temp.shape[1])
    print(_temp[_temp==0].shape)
    print(_temp[_temp==1].shape)
    print(_temp[_temp==2].shape)
    print(_temp[_temp==3].shape)
    print(_temp[_temp==4].shape)
    print(_temp[_temp==5].shape)
    print(_temp[_temp==6].shape)
    print(_temp[_temp==7].shape)
    print(_temp[_temp==8].shape)
    print(_temp[_temp==9].shape)
    print(_temp[_temp==10].shape)
    
    """
    print(np.unique(_temp))
    
    print(np.unique(_temp, return_counts=True))
    """
    
    cols, rows, n_bands = array_to_write[:dim_x, :dim_y, :].shape
    
    outFileName = input_identifier + "_MANGROVE_PREDICTION.tiff"
    driver = gdal.GetDriverByName("GTiff")

    outdata = driver.Create(outFileName, rows, cols, n_bands, gdal.GDT_Byte, options=['COMPRESS=DEFLATE','PREDICTOR=1'])
    
    
    outdata.SetGeoTransform(transf)##sets same geotransform as input
    outdata.SetProjection(proj)##sets same projection as input
    
    for band in range(0, n_bands):
        outdata.GetRasterBand(band+1).WriteArray(_temp[:dim_x, :dim_y, band])
    
    # https://gis.stackexchange.com/questions/325615/store-geotiff-with-color-table-python/325751
    # https://gis.stackexchange.com/questions/352495/converted-vector-to-raster-file-is-black-and-white-in-colour-gdal-rasterize
    # https://github.com/mapbox/rasterio/issues/100
    band = outdata.GetRasterBand(1)
    band.SetNoDataValue(0.0)
    # create color table
    colors = gdal.ColorTable()

    # set color for each value
    #colors.SetColorEntry(0, (215, 25, 28, 0))
    #colors.SetColorEntry(1, (231, 84, 55, 0))
    #colors.SetColorEntry(2, (246, 144, 83, 0))
    colors.SetColorEntry(3, (254, 190, 116, 0))
    colors.SetColorEntry(4, (255, 223, 154, 0))
    colors.SetColorEntry(5, (255, 255, 191, 0))
    colors.SetColorEntry(6, (222, 242, 180, 0))
    colors.SetColorEntry(7, (188, 228, 170, 0))
    colors.SetColorEntry(8, (145, 203, 169, 0))
    colors.SetColorEntry(9, (94, 167, 177, 0))
    colors.SetColorEntry(10, (43, 131, 186, 0))

    ## set color table and color interpretation ##
    
    # set color table for the "png" image
    band.SetRasterColorTable(colors)
    band.SetRasterColorInterpretation(gdal.GCI_SaturationBand)
    
    # set color table for the QGIS intrepertation 
    band.SetColorTable(colors)
    
    band.DeleteNoDataValue()


    stats = band.GetStatistics(0,1)

    
    outdata.FlushCache() ##saves to disk!!
    outdata = None
    colors = None
    band = None
    print("image created")  

#### Gather the data

In [98]:
array_t, transf, proj = read_bands_to_array(my_data_path)

/workspace/data/S2B_MSIL2A_20190609T112119_N0212_R037_T28PDT_20190609T144134/S2B_MSIL2A_20190609T112119_N0212_R037_T28PDT_20190609T144134.SAFE/manifest.safe

GeoTransform: (399960.0, 10.0, 0.0, 1300020.0, 0.0, -10.0)
Projection: PROJCS["WGS 84 / UTM zone 28N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32628"]]
Size is 10980 x 10980 x 1
New band: /workspace/data/S2B_MSIL2A_20190609T112119_N0212_R037_T28PDT_20190609T144134/S2B_MSIL2A_20190609T112119_N0212_R037_T28PDT_20190609T14413

In [99]:
array_t = array_normalizer(array_t.astype(np.float32))

In [100]:
array_t2 = create_indices(array_t[:, :, :])


Create indices
MNDWI created
NDVI created
MMRI created


In [101]:
dim_x, dim_y, bands = array_t2.shape
tile_size = 512
tiles_array, n_tiles_x, n_tiles_y, pad_x, pad_y  = array_cutter(array_t2, tile_size, tile_size)

Horizontal tiles:  22
Vertical tiles:  22
Total tiles:  484
pad x:  284
pad y:  284
Size of final structure:  (484, 512, 512, 3)


#### Run the model

In [102]:
model_name = 'mangrove_model_V1.hdf5'
LOCAL_MODEL_PATH = "./{}".format(model_name)
APP_MODEL_PATH = "/application/notebook/libexec/{}".format(model_name)

if os.path.isfile(LOCAL_MODEL_PATH):
    mangrove_model = models.load_model(LOCAL_MODEL_PATH, compile=False)
elif os.path.isfile(APP_MODEL_PATH):
    mangrove_model = models.load_model(APP_MODEL_PATH, compile=False)
else:
    sys.exit()

In [103]:
score = mangrove_model.predict(tiles_array[:, :, :, :], verbose=1, use_multiprocessing=True, max_queue_size=10, workers = 4)
print("Prediction completed")

484/484 [==============================] - 5363s 11s/sample
Prediction completed


#### Reconstruct the image

In [104]:
print(score.shape)
print(score[:,:,:,2].shape)

(484, 512, 512, 4)
(484, 512, 512)


In [105]:
dim_x, dim_y, _ = array_t.shape
n_bands = 1
#Note: only band 3 is important
create_image(my_data_path, score[:,:,:,2], n_tiles_x, n_tiles_y, tile_size, dim_x, dim_y, n_bands, pad_x, pad_y, transf, proj)


(10980, 10980, 1)
size:  126877696
(0,)
(0,)
(0,)
(126865819,)
(5522,)
(3225,)
(1861,)
(876,)
(349,)
(44,)
(0,)
image created


In [119]:
#translate to the EPSG:4326
outFileName = input_identifier + "_MANGROVE_PREDICTION.tiff"

ds = gdal.Warp(srcDSOrSrcDSTab  = outFileName, destNameOrDestDS = "_temp.tiff", dstSRS='EPSG:4326')

_temp_img = gdal.Open("_temp.tiff")
transf = _temp_img.GetGeoTransform()
proj = _temp_img.GetProjection()
    
print()
print("GeoTransform: {}".format(transf))
print("Projection: {}".format(proj))     
print("Size is {} x {} x {}".format(_temp_img.RasterXSize,
                                    _temp_img.RasterYSize,
                                    _temp_img.RasterCount))
ds=None
os.remove("_temp.tiff")


GeoTransform: (-15.918140902383238, 9.10303122186868e-05, 0.0, 11.760044025669968, 0.0, -9.10303122186868e-05)
Projection: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Size is 11070 x 10923 x 1


#### Create proprieties file

In [120]:
print("Create proprities file")
# get the time stamp from the name of the file
t_time = input_identifier.split("_")
tt1 = t_time[2][:8]
tt1 = tt1[:4] + "-" + tt1[4:6] + "-" + tt1[6:]
tt1 = tt1 + "T00:00:00Z/" + tt1 + "T23:59:59Z"
print(tt1)

Create proprities file
2019-06-09T00:00:00Z/2019-06-09T23:59:59Z


In [121]:
# get the necessary geotransform variables
minx = transf[0]
maxy = transf[3]
maxx = minx + transf[1] * dim_x
miny = maxy + transf[5] * dim_y
    
wktstr = 'POLYGON(({0} {1}, {2} {1}, {2} {3}, {0} {3}, {0} {1}))'.format(minx, maxy, maxx, miny)

In [122]:
title = "Output " + input_identifier + "_MANGROVE_PREDICTION.tiff"
output_name = input_identifier + "_MANGROVE_PREDICTION.tiff"

with open(output_name + '.properties', 'w') as file:
    file.write('title=%s\n' % title)
    file.write('date=%s\n' % (tt1))
    file.write('geometry=%s' % (wktstr))

#### Create the aux.xml file

In [109]:
file_name='template.tiff.aux.xml'

LOCAL_TEMPLATE_PATH = "./{}".format(file_name)
APP_TEMPLATE_PATH = "/application/notebook/libexec/{}".format(file_name)

if os.path.isfile(LOCAL_TEMPLATE_PATH):
    file_name = LOCAL_TEMPLATE_PATH
elif os.path.isfile(APP_TEMPLATE_PATH):
    file_name = APP_TEMPLATE_PATH
else:
    sys.exit()

f = open(file_name, "r")
xml_content = None
try: 
    xml_content = f.read()
except Exception as e:
    print("could not read xml")
    print(e)
    sys.exit(1)
finally:
    f.close
        
print(xml_content)

f = open(input_identifier + "_MANGROVE_PREDICTION.tiff.aux.xml", "w")
f.write(xml_content)
f.close()

<PAMDataset>
  <PAMRasterBand band="1">
    <CategoryNames>
      <Category>0.no Mangrove Probability</Category>
      <Category>1.no Mangrove Probability</Category>
      <Category>2.no Mangrove Probability</Category>
      <Category>3.Very Very Low Mangrove Probability</Category>
      <Category>4.Very Low Mangrove Probability</Category>
      <Category>5.Low Mangrove Probability</Category>
      <Category>6.Medium Low Mangrove Probability</Category>
      <Category>7.Medium High Mangrove Probability</Category>
      <Category>8.High Mangrove Probability</Category>
      <Category>9.Very High Mangrove Probability</Category>
      <Category>10.Very Very High Mangrove Probability</Category>
    </CategoryNames>
  </PAMRasterBand>
</PAMDataset>


### <a name="license">License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.